# Loan Default Prediction
## Team Members:
* Harish Puvvada
* Vamsi Mohan Ramineedi

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv(os.getenv("FDSDATA")+'LoanStats_2017Q1.csv',low_memory=False, skiprows=1)
df2 = pd.read_csv(os.getenv("FDSDATA")+'LoanStats_2017Q2.csv',low_memory=False, skiprows=1)
df3 = pd.read_csv(os.getenv("FDSDATA")+'LoanStats_2017Q3.csv',low_memory=False, skiprows=1)

In [3]:
# Dataframes = [Q1data,Q2data,Q3data]
dataset = pd.concat([df1, df2, df3])

In [4]:
dataset.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,5000.0,36 months,25.49%,200.10,E,E4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6800.0,6800.0,6800.0,36 months,13.49%,230.73,C,C2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,15400.0,15400.0,15400.0,60 months,11.39%,337.84,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,6000.0,6000.0,6000.0,36 months,11.44%,197.69,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,15000.0,15000.0,15000.0,36 months,5.32%,451.73,A,A1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dataset.shape


(324937, 145)

In [6]:
dataset.columns


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=145)

In [ ]:
dataset